In [1]:
from config import DATA_DIR
from utils import sample_series, save_df, read_df
import pandas as pd
import nltk
import re
from unidecode import unidecode

Abrindo os dados originais parseados

In [4]:
df: pd.DataFrame = read_df('dados_originais_sugestoes.csv')

In [5]:
df.sample(3)

,excerto,categoria
3305,Combater as agressões contra crianças e adoles...,Direitos Humanos e Cidadania
103,"1. A meta 3, aqui por Jardim Julieta, de...",Saúde
353,Finalmente a questão alimentação é gravíssima ...,Assistência Social


Limpezas simples do texto -- tirar maiusculas etc.

In [6]:
df.rename({'excerto' : 'texto_original'}, axis=1, inplace=True)

In [7]:
df['texto_limpo'] = df['texto_original'].str.lower().str.strip('/n')


In [6]:
print(sample_series(df['texto_limpo']))

meta 37 - iniciativa j - revisar - incluir a implantação de faixa elevada de pedestre ao longo da calçada verde da rua manuel dos reis araújo no jardim marajoara para melhoria da segurança dos munícipes que praticam caminhadas e diversas outras atividades físicas e de lazer no local.


Baixando as stopwords para limpar os textos

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/h-pgy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [10]:
stopwords[:3]

['a', 'à', 'ao']

In [11]:
type(stopwords)

list

In [12]:
pdm_stopwords: list[str] = ['programa de metas',
                            'pdm',
                            'audiência',
                            'audiencia',
                            'boa noite',
                            'bom dia',
                            'boa tarde',
                            'meta',
                            'olá',
                            'iniciativa'
                        ]

In [13]:
stopwords.extend(pdm_stopwords)

stopwords = set(stopwords)

tokenizando o texto

In [14]:
from nltk.tokenize import word_tokenize

In [15]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/h-pgy/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [16]:
df['tokens'] = df['texto_limpo'].apply(word_tokenize)

In [17]:
print(sample_series(df['tokens']))

['em', 'relação', 'às', 'construções', 'de', 'unidades', ',', 'aqui', 'da', 'região', 'e', 'o', 'ubs', ',', 'nós', 'temos', 'banda', 'coelho', 'e', 'morais', 'na', 'rua', 'sepetiba', 'e', 'tem', 'dinheiro', 'do', 'bid', 'para', 'construção', 'e', 'tem', 'projeto', '.']


In [18]:
def remove_stopwords(tokens:str, stopwords:set[str]=stopwords)->list[str]:

    filtered: list[str]=[token for token in tokens
                         if token not in stopwords]
    
    return filtered





In [19]:
df['tokens_clean'] = df['tokens'].apply(remove_stopwords)

In [20]:
print(sample_series(df['tokens_clean']))

['relação', 'construções', 'unidades', ',', 'aqui', 'região', 'ubs', ',', 'banda', 'coelho', 'morais', 'rua', 'sepetiba', 'dinheiro', 'bid', 'construção', 'projeto', '.']


Remover caracteres especiais e pontuacao

In [21]:
def remover_char_especiais(tokens:list[str])->list[str]:

    #tudo o que não são caracteres alphanumericos
    patt = r'[^\w]'
    limpo:list[str] = []
    for token in tokens:
        token = re.sub(patt, '', token)
        if token:
            limpo.append(token)
    return limpo

In [22]:
remover_char_especiais(['ontem', '123', '@'])

['ontem', '123']

In [23]:
df['tokens_clean'] = df['tokens_clean'].apply(remover_char_especiais)

In [24]:
print(sample_series(df['tokens_clean']))

['relação', 'construções', 'unidades', 'aqui', 'região', 'ubs', 'banda', 'coelho', 'morais', 'rua', 'sepetiba', 'dinheiro', 'bid', 'construção', 'projeto']


Rmover os números - são referência às metas, podem enviesar o modelo.

In [25]:
'214'.isdigit()

True

In [26]:
def remove_numbers(tokens:list[str])->list[str]:

    cleaned_tokens: list[str] = []

    for token in tokens:
        while token.startswith('0'):
            token=token.lstrip('0')
        
        if not token.isdigit():
            cleaned_tokens.append(token)

    return cleaned_tokens

In [27]:
df['tokens_clean'] = df['tokens_clean'].apply(remove_numbers)

In [28]:
print(sample_series(df['tokens_clean']))

['relação', 'construções', 'unidades', 'aqui', 'região', 'ubs', 'banda', 'coelho', 'morais', 'rua', 'sepetiba', 'dinheiro', 'bid', 'construção', 'projeto']


remover acentos

In [29]:
def remover_acentos(tokens:list[str])->list[str]:

    return [unidecode(token) for token in tokens]

In [30]:
df['tokens_clean'] = df['tokens_clean'].apply(remover_acentos)

In [31]:
print(sample_series(df['tokens_clean']))

['relacao', 'construcoes', 'unidades', 'aqui', 'regiao', 'ubs', 'banda', 'coelho', 'morais', 'rua', 'sepetiba', 'dinheiro', 'bid', 'construcao', 'projeto']


In [32]:
save_df(df, 'dados_tokenizados.csv')

'../data/dados_tokenizados.csv'